In [1]:
from pyaci import Node, options

## Turn on Logging
Some of the `INFO` level log messages might help understanding the WebSocket control flow. If you are interested further, you can uncomment the line that increases the verbosity of messages.

In [2]:
import logging
logging.basicConfig(level=logging.INFO)
# logging.getLogger('pyaci').setLevel(logging.DEBUG)

## Establish API Connection

In [3]:
n = Node('https://praveek6-bld.insieme.local:7000')
n.methods.Login('admin').POST()

Enter admin password: ········


<Response [200]>

## Start the WebSocket Listener

This will initiate a connection to the APIC WebSocket URL, and will block until the connection is ready.

In [4]:
n.startWsListener()

INFO:pyaci.core:Establishing WebSocket connection to wss://praveek6-bld.insieme.local:7000/socketDAAAAAAAAAAAAAAAAAAAAIblE02bMwLAS2bLrvMUZkgJoyZ2u1vfIusOGVJxYBRslKYRk/+L9n5Gu8v0x7Dqde3xBr5QYXpW4jz20M5iyoxhqGknNFUCSy3lmVlchQMXNuMC5O6VpdBDdNPOcYwY4kdODyL/OtlhHlU/lsvuvES4fNHL7n2j6FllRh/gyzGjnCbyYRJU0bTbaiEOKCrBig==
INFO:pyaci.core:Waiting for the WebSocket connection to open
INFO:pyaci.core:Opened WebSocket connection


## Subscribe to Query

You can subscribe to any `GET()` request by passing in the option `options.subscribe`. When this option is enabled, the result will be a tuple. The first field in the tuple is the list of resulting MOs. The second field in the tuple will be the subscription ID. This subscription ID is later used to query for MOs that arrived for that subscription.

In [5]:
tenants, subscriptionId = n.methods.ResolveClass('fvTenant').GET(**options.subscribe)

## Checking if MOs Available to Read

Node has a method `hasWsMo()` that will return `True` if there are any notification MOs, that can be read. You can use this to manually poll.

In [6]:
n.hasWsMo(subscriptionId)

False

## Triggering a Notification

Let's create a tenant, that will trigger a notification for the previous subscription.

In [7]:
n.mit.polUni().fvTenant('acme').POST()

<Response [200]>

## Waiting for MOs

Alternately, one could use `waitForWsMo()` method to block until a MO is available to read for that subscription.

In [8]:
n.waitForWsMo(subscriptionId)

INFO:pyaci.core:Waiting for the WebSocket MOs


In [9]:
n.hasWsMo(subscriptionId)

True

## Pop MO from the Notification Queue

In [10]:
print n.popWsMo(subscriptionId).Json

{
  "fvTenant": {
    "attributes": {
      "childAction": "",
      "descr": "",
      "dn": "uni/tn-acme",
      "extMngdBy": "",
      "lcOwn": "local",
      "modTs": "2017-11-15T07:52:26.292+00:00",
      "monPolDn": "uni/tn-common/monepg-default",
      "name": "acme",
      "nameAlias": "",
      "ownerKey": "",
      "ownerTag": "",
      "rn": "",
      "status": "created",
      "uid": "15374"
    }
  }
}
